In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
#Read the csv file
dataset = pd.read_csv('data/2_resampled_heart_attack_data.csv')
# dataset

In [3]:
# Splitting the dataset into features and target variable
X = dataset.drop('Heart Attack Risk', axis=1)
y = dataset['Heart Attack Risk']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data
X_test_scaled = scaler.transform(X_test)

In [6]:
# Initialize the SVM model
svm_model = SVC()

In [7]:
# Define the hyperparameter grid for the SVM model
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [0.001, 0.01, 0.1, 1],
    'degree': [2, 3, 4]
}

In [8]:
# Initialize the grid search with cross-validation
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='roc_auc')

In [9]:
# Fit the grid search to the scaled training data
grid_search.fit(X_train_scaled, y_train)

KeyboardInterrupt: 

In [ ]:
# Get the best hyperparameters
best_params = grid_search.best_params_
print(f'Best hyperparameters: {best_params}')

In [ ]:
# Get the best estimator (model with best hyperparameters)
best_svm_model = grid_search.best_estimator_

In [ ]:
# Make predictions with the best model on the scaled test data
y_pred = best_svm_model.predict(X_test_scaled)

In [ ]:
# Evaluate the model's performance
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

In [ ]:
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)


In [ ]:
# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the best SVM model with scaling: {roc_auc:.2f}')

In [ ]:
# ROC Curve and AUC-ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC-ROC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], 'k--')  # Dashed diagonal
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()